In [46]:
# import all the various libraries
import pandas as pd
import json
from zipfile import ZipFile
from bs4 import BeautifulSoup
from azure.cosmos import CosmosClient
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import os

In [47]:
os.environ['COSMOS_URI'] = 'https://localhost:8081'
os.environ['COSMOS_KEY'] = 'C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw=='

In [50]:
json_url = open('D:\GitHub\sbsa_excel_to_cosmos_etl\AlertsDB\AlertsContainer-data.json', encoding='utf-8')
json_data = json.load(json_url)

for i in range(len(json_data)):
    t = json_data[i]
    try:
        del t['_rid']
        del t['_self']
        del t['_etag']
        del t['_ts']
        del t['_attachments']
    except:
        continue

json_data



[{'id': 'e4037333-9567-4982-8be1-cd893ddaec05',
  'Name': 'CISCA Circular No 31',
  'WebsiteName': 'https://www.fsca.co.za/Regulatory%20Frameworks/Pages/Circulars.aspx',
  'DateExtracted': '2021-11-04T20:27:51.143923Z',
  'Jurisdiction': 'London',
  'Regulator': 'Financial Sector Conduct Authority',
  'PageName': 'Circulars',
  'PublicationDate': 'None',
  'Description': '',
  'DocumentURL': 'https://www.fsca.co.za/Regulatory%20Frameworks/Temp/CISCA%20Circular%20No%2031.pdf',
  'BlobStorageURL': '2021-11-04T20-27-49_CISCA Circular No 31_1.pdf',
  'Status': 11,
  'FileHash': '90F40CFCCC938FB3AD27194B6E35F4A1D39ECF1B2C7AA0EB4FEFD2583A9C6B57',
  'AlertSourceId': '2082a64b-4586-4d7d-9010-8904eeeeaecb',
  'RTAReviewRequired': False,
  'AlertFlagReason': 'None',
  'OriginalText': 'None',
  'CleanedText': 'None',
  'TokenizedText': 'None',
  'DocFilename': 'None',
  'CosineSimilarityScore': 'None',
  'SoftCosineSimilarityScore': 'None',
  'CosineFilenameSimilarTo': 'None',
  'SoftCosineFilena

In [51]:
# instantiate the cosmos db client
url = os.environ['COSMOS_URI']
key = os.environ['COSMOS_KEY']

client = CosmosClient(url, credential=key)

db = client.get_database_client('AlertsDB')

container = db.get_container_client('AlertsContainer')

for i in range(len(json_data)):
    container.create_item(body=json_data[i])

C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\A